In [15]:
!pip install git+https://github.com/openai/CLIP.git qdrant-client pytesseract gradio opencv-python pillow scikit-learn symspellpy openai -q
!apt-get install tesseract-ocr -qq


  Preparing metadata (setup.py) ... done


In [ ]:
!pip install git+https://github.com/openai/CLIP.git qdrant-client pytesseract gradio opencv-python pillow scikit-learn symspellpy openai -q
!apt-get install tesseract-ocr -qq


  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, uuid, cv2, torch, clip, numpy as np
import pytesseract
import gradio as gr

from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

import pkg_resources

from google.colab import userdata
from openai import OpenAI


In [ ]:
API_KEY = userdata.get("API_KEY")
BASE_URL = userdata.get("BASE_URL")

ai_client = OpenAI(api_key=API_KEY, base_url=BASE_URL) if BASE_URL else OpenAI(api_key=API_KEY)

print("✅ OpenAI connected")


✅ OpenAI connected


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

print("CLIP running on:", device)

qdrant_client = QdrantClient(":memory:")
DIM = 512


CLIP running on: cuda


In [ ]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, 0, 1)

def clean_query(text):
    if not text:
        return ""
    text = text.lower().strip()
    suggestion = sym_spell.lookup_compound(text, max_edit_distance=2)
    return suggestion[0].term if suggestion else text


In [ ]:
DATA_ROOT = "/content/drive/MyDrive/clustered_images"

clusters = {}

for item_type in os.listdir(DATA_ROOT):
    type_path = os.path.join(DATA_ROOT, item_type)
    if not os.path.isdir(type_path):
        continue

    for metal in os.listdir(type_path):
        metal_path = os.path.join(type_path, metal)
        if not os.path.isdir(metal_path):
            continue

        name = f"{item_type}_{metal}"

        images = [
            os.path.join(metal_path, f)
            for f in os.listdir(metal_path)
            if f.lower().endswith((".jpg",".png",".jpeg",".webp"))
        ]

        clusters[name] = images

for k,v in clusters.items():
    print(k, "→", len(v))


ring_gold → 52
ring_silver → 17
ring_copper → 4
necklace_gold → 9
necklace_silver → 0
necklace_copper → 0


In [ ]:
def get_image_emb(path):
    image = Image.open(path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = clip_model.encode_image(image_input)

    return (emb / emb.norm(dim=-1, keepdim=True))[0]

def get_text_emb(text):
    tokens = clip.tokenize([text]).to(device)

    with torch.no_grad():
        emb = clip_model.encode_text(tokens)

    return (emb / emb.norm(dim=-1, keepdim=True))[0]


In [ ]:
def ocr_text(path):
    try:
        img = cv2.imread(path)
        if img is None:
            return ""
        return pytesseract.image_to_string(img).strip()
    except:
        return ""

def fuse_emb(img_emb, txt_emb, alpha=0.7):
    fused = alpha * img_emb + (1 - alpha) * txt_emb
    return fused / fused.norm()


In [ ]:
def index_drive():

    for cluster_name, images in clusters.items():

        qdrant_client.recreate_collection(
            collection_name=cluster_name,
            vectors_config=VectorParams(size=DIM, distance=Distance.COSINE)
        )

        points = []

        for path in images:
            emb = get_image_emb(path)

            points.append(
                PointStruct(
                    id=str(uuid.uuid4()),
                    vector=emb.cpu().numpy().tolist(),
                    payload={"path": path}
                )
            )

        qdrant_client.upsert(collection_name=cluster_name, points=points)

        print(f"✅ Indexed {cluster_name}: {len(points)} images")

index_drive()


/tmp/ipython-input-1336531758.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


✅ Indexed ring_gold: 52 images
✅ Indexed ring_silver: 17 images
✅ Indexed ring_copper: 4 images
✅ Indexed necklace_gold: 9 images
✅ Indexed necklace_silver: 0 images
✅ Indexed necklace_copper: 0 images


In [ ]:
METALS = ["gold", "silver", "copper"]
TYPES = ["ring", "necklace"]

def route_metal(text):
    if not text:
        return None
    for m in METALS:
        if m in text:
            return m
    return None

def route_type(text):
    if not text:
        return None
    for t in TYPES:
        if t in text:
            return t
    return None


In [ ]:
GEMS = ["ruby", "diamond", "emerald", "sapphire"]

def route_gem(text):
    if not text:
        return None
    for g in GEMS:
        if g in text:
            return g
    return None


In [ ]:
def search_all(emb, metal=None, item_type=None, topk=20):

    hits_all = []

    for collection in qdrant_client.get_collections().collections:

        name = collection.name

        if metal and metal not in name:
            continue

        if item_type and item_type not in name:
            continue

        hits = qdrant_client.query_points(
            collection_name=name,
            query=emb.cpu().numpy().tolist(),
            limit=topk,
            with_vectors=True
        )

        hits_all.extend(hits.points)

    return hits_all


In [ ]:
def detect_gem_color(path):

    img = cv2.imread(path)
    if img is None:
        return "unknown"

    img = cv2.resize(img, (200, 200))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # detect white (diamond)
    white_mask = cv2.inRange(hsv, (0, 0, 200), (180, 40, 255))
    white_ratio = np.sum(white_mask > 0) / white_mask.size

    # if mostly white → diamond
    if white_ratio > 0.12:
        return "diamond"

    # otherwise any colored stone → ruby
    return "ruby"


In [ ]:
def run_query(text=None, image_path=None):

    text = clean_query(text)

    if text and not image_path:
        emb = get_text_emb(text)

    elif image_path and not text:
        emb = get_image_emb(image_path)

    else:
        ocr = ocr_text(image_path) if not text or len(text) < 3 else ""
        combined = (text or "") + " " + ocr

        img_emb = get_image_emb(image_path)
        txt_emb = get_text_emb(combined)
        emb = fuse_emb(img_emb, txt_emb)

    metal = route_metal(text)
    item_type = route_type(text)
    gem = route_gem(text)

    hits = search_all(emb, metal, item_type)

    if not hits:
        return [], "❌ Requested product not available sir 😔"

    strict_bucket = []
    fallback_bucket = []

    query_vec = emb.cpu().numpy()

    for h in hits:

        path = h.payload["path"]
        detected = detect_gem_color(path)

        vec = np.array(h.vector)
        score = cosine_similarity([query_vec], [vec])[0][0]

        if gem and detected == gem:
            strict_bucket.append((score, path))
        else:
            fallback_bucket.append((score, path))

    # sort buckets
    strict_bucket.sort(reverse=True)
    fallback_bucket.sort(reverse=True)

    # 🔥 fallback protection
    if strict_bucket:
        scored = strict_bucket + fallback_bucket
    else:
        scored = fallback_bucket   # never return empty

    top = scored[:6]

    results = [p for _, p in top]
    score = top[0][0]

    review = salesman_review_ai(results[0], text)
    review += f"\n\nMatch score: {score:.2f}"

    return results, review


In [ ]:
def salesman_review_ai(path, query_text):

    parts = path.split(os.sep)
    metal = parts[-2]
    item_type = parts[-3]

    query_text = query_text if query_text else "image reference"

    prompt = f"""
You are a charming Tamil jewelry salesman.

Customer request: {query_text}
Matched item: {metal} {item_type}

Speak in natural Tanglish.
Use 3–4 bullet points.
Mention style, premium feel, occasion.
Friendly tone.
"""

    r = ai_client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role":"user","content":prompt}]
    )

    return r.choices[0].message.content


In [ ]:
def gradio_search(text, image):
    image_path = image if image else None
    results, review = run_query(text, image_path)
    return results, review

import nest_asyncio
nest_asyncio.apply()

demo = gr.Interface(
    fn=gradio_search,
    inputs=[gr.Textbox(), gr.File(type="filepath")],
    outputs=[gr.Gallery(), gr.Textbox()],
    title="💎 AI Jewelry Search"
)

demo.queue(False)
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd4a0b33677c2e9f5d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os, uuid, cv2, torch, clip, numpy as np
import pytesseract
import gradio as gr

from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

from symspellpy import SymSpell
import pkg_resources

from google.colab import userdata
from openai import OpenAI


In [17]:
API_KEY = userdata.get("API_KEY")
BASE_URL = userdata.get("BASE_URL")

ai_client = OpenAI(api_key=API_KEY, base_url=BASE_URL) if BASE_URL else OpenAI(api_key=API_KEY)

print("✅ OpenAI connected")


✅ OpenAI connected


In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

print("CLIP running on:", device)

qdrant_client = QdrantClient(":memory:")
DIM = 512


CLIP running on: cuda


In [29]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, 0, 1)

def clean_query(text):
    if not text:
        return ""
    text = text.lower().strip()
    suggestion = sym_spell.lookup_compound(text, max_edit_distance=2)
    return suggestion[0].term if suggestion else text


In [30]:
DATA_ROOT = "/content/drive/MyDrive/clustered_images"

clusters = {}

for item_type in os.listdir(DATA_ROOT):
    type_path = os.path.join(DATA_ROOT, item_type)
    if not os.path.isdir(type_path):
        continue

    for metal in os.listdir(type_path):
        metal_path = os.path.join(type_path, metal)
        if not os.path.isdir(metal_path):
            continue

        name = f"{item_type}_{metal}"

        images = [
            os.path.join(metal_path, f)
            for f in os.listdir(metal_path)
            if f.lower().endswith((".jpg",".png",".jpeg",".webp"))
        ]

        clusters[name] = images

for k,v in clusters.items():
    print(k, "→", len(v))


ring_gold → 52
ring_silver → 17
ring_copper → 4
necklace_gold → 9
necklace_silver → 0
necklace_copper → 0


In [31]:
def get_image_emb(path):
    image = Image.open(path).convert("RGB")
    image_input = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        emb = clip_model.encode_image(image_input)

    return (emb / emb.norm(dim=-1, keepdim=True))[0]

def get_text_emb(text):
    tokens = clip.tokenize([text]).to(device)

    with torch.no_grad():
        emb = clip_model.encode_text(tokens)

    return (emb / emb.norm(dim=-1, keepdim=True))[0]


In [32]:
def ocr_text(path):
    try:
        img = cv2.imread(path)
        if img is None:
            return ""
        return pytesseract.image_to_string(img).strip()
    except:
        return ""

def fuse_emb(img_emb, txt_emb, alpha=0.7):
    fused = alpha * img_emb + (1 - alpha) * txt_emb
    return fused / fused.norm()


In [33]:
def index_drive():

    for cluster_name, images in clusters.items():

        qdrant_client.recreate_collection(
            collection_name=cluster_name,
            vectors_config=VectorParams(size=DIM, distance=Distance.COSINE)
        )

        points = []

        for path in images:
            emb = get_image_emb(path)

            points.append(
                PointStruct(
                    id=str(uuid.uuid4()),
                    vector=emb.cpu().numpy().tolist(),
                    payload={"path": path}
                )
            )

        qdrant_client.upsert(collection_name=cluster_name, points=points)

        print(f"✅ Indexed {cluster_name}: {len(points)} images")

index_drive()


/tmp/ipython-input-1336531758.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


✅ Indexed ring_gold: 52 images
✅ Indexed ring_silver: 17 images
✅ Indexed ring_copper: 4 images
✅ Indexed necklace_gold: 9 images
✅ Indexed necklace_silver: 0 images
✅ Indexed necklace_copper: 0 images


In [42]:
METALS = ["gold", "silver", "copper"]
TYPES = ["ring", "necklace"]

def route_metal(text):
    if not text:
        return None
    for m in METALS:
        if m in text:
            return m
    return None

def route_type(text):
    if not text:
        return None
    for t in TYPES:
        if t in text:
            return t
    return None


In [43]:
GEMS = ["ruby", "diamond", "emerald", "sapphire"]

def route_gem(text):
    if not text:
        return None
    for g in GEMS:
        if g in text:
            return g
    return None


In [53]:
def search_all(emb, metal=None, item_type=None, topk=20):

    hits_all = []

    for collection in qdrant_client.get_collections().collections:

        name = collection.name

        if metal and metal not in name:
            continue

        if item_type and item_type not in name:
            continue

        hits = qdrant_client.query_points(
            collection_name=name,
            query=emb.cpu().numpy().tolist(),
            limit=topk,
            with_vectors=True
        )

        hits_all.extend(hits.points)

    return hits_all


In [69]:
def detect_gem_color(path):

    img = cv2.imread(path)
    if img is None:
        return "unknown"

    img = cv2.resize(img, (200, 200))
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # detect white (diamond)
    white_mask = cv2.inRange(hsv, (0, 0, 200), (180, 40, 255))
    white_ratio = np.sum(white_mask > 0) / white_mask.size

    # if mostly white → diamond
    if white_ratio > 0.12:
        return "diamond"

    # otherwise any colored stone → ruby
    return "ruby"


In [73]:
def run_query(text=None, image_path=None):

    text = clean_query(text)

    if text and not image_path:
        emb = get_text_emb(text)

    elif image_path and not text:
        emb = get_image_emb(image_path)

    else:
        ocr = ocr_text(image_path) if not text or len(text) < 3 else ""
        combined = (text or "") + " " + ocr

        img_emb = get_image_emb(image_path)
        txt_emb = get_text_emb(combined)
        emb = fuse_emb(img_emb, txt_emb)

    metal = route_metal(text)
    item_type = route_type(text)
    gem = route_gem(text)

    hits = search_all(emb, metal, item_type)

    if not hits:
        return [], "❌ Requested product not available sir 😔"

    strict_bucket = []
    fallback_bucket = []

    query_vec = emb.cpu().numpy()

    for h in hits:

        path = h.payload["path"]
        detected = detect_gem_color(path)

        vec = np.array(h.vector)
        score = cosine_similarity([query_vec], [vec])[0][0]

        if gem and detected == gem:
            strict_bucket.append((score, path))
        else:
            fallback_bucket.append((score, path))

    # sort buckets
    strict_bucket.sort(reverse=True)
    fallback_bucket.sort(reverse=True)

    # 🔥 fallback protection
    if strict_bucket:
        scored = strict_bucket + fallback_bucket
    else:
        scored = fallback_bucket   # never return empty

    top = scored[:6]

    results = [p for _, p in top]
    score = top[0][0]

    review = salesman_review_ai(results[0], text)
    review += f"\n\nMatch score: {score:.2f}"

    return results, review


In [71]:
def salesman_review_ai(path, query_text):

    parts = path.split(os.sep)
    metal = parts[-2]
    item_type = parts[-3]

    query_text = query_text if query_text else "image reference"

    prompt = f"""
You are a charming Tamil jewelry salesman.

Customer request: {query_text}
Matched item: {metal} {item_type}

Speak in natural Tanglish.
Use 3–4 bullet points.
Mention style, premium feel, occasion.
Friendly tone.
"""

    r = ai_client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role":"user","content":prompt}]
    )

    return r.choices[0].message.content


In [74]:
def gradio_search(text, image):
    image_path = image if image else None
    results, review = run_query(text, image_path)
    return results, review

import nest_asyncio
nest_asyncio.apply()

demo = gr.Interface(
    fn=gradio_search,
    inputs=[gr.Textbox(), gr.File(type="filepath")],
    outputs=[gr.Gallery(), gr.Textbox()],
    title="💎 AI Jewelry Search"
)

demo.queue(False)
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dd4a0b33677c2e9f5d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
